# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Create Dataframe from cities file generated from WeatherPy

file = os.path.join("..","WeatherPy","output_data","cities.csv")

cities_df = pd.read_csv(file)

cities_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 563 entries, 0 to 562
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  563 non-null    int64  
 1   City ID     563 non-null    int64  
 2   City        563 non-null    object 
 3   Country     557 non-null    object 
 4   Date        563 non-null    int64  
 5   Cloudiness  563 non-null    int64  
 6   Humidity    563 non-null    int64  
 7   Latitude    563 non-null    float64
 8   Longitude   563 non-null    float64
 9   Max Temp    563 non-null    float64
 10  Wind Speed  563 non-null    float64
dtypes: float64(4), int64(5), object(2)
memory usage: 48.5+ KB


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Configure gmaps
gmaps.configure(g_key)

# Use Lat/Long from cities_df to store as locations

locations = cities_df[['Latitude','Longitude']]

# Store humidity as weight

weight = cities_df['Humidity']

# Store Heatmap in fig

fig = gmaps.figure()

# Create heat layer

heat_layer = gmaps.heatmap_layer(locations, weights=weight,dissipating = False,max_intensity = 10,point_radius=1)

fig.add_layer(heat_layer)

fig



Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:

# Convert Kelvin to Fahrenheit
cities_df['Max Temp'] =(cities_df['Max Temp'] -273.15)*9/5 + 32

In [5]:
# Narrow down DataFrame (Max Temp>70 but <80, Wind Speed <10, Cloudiness = 0, drop all the do not contain those values)


ideal_df = cities_df.loc[(cities_df['Max Temp']>70) & 
                             (cities_df['Max Temp']<80) & 
                             (cities_df['Cloudiness']==0) &
                             (cities_df['Wind Speed']<10),
                             :]



ideal_df.head()


,Unnamed: 0,City ID,City,Country,Date,Cloudiness,Humidity,Latitude,Longitude,Max Temp,Wind Speed
8,8,3356832,Henties Bay,NaN,1592446139,0,31,-22.12,14.28,73.994,0.89
20,20,6620339,Karratha,AU,1592446142,0,35,-20.74,116.85,78.674,2.29
51,51,2065594,Mount Isa,AU,1592446150,0,41,-20.73,139.50,78.800,6.20
114,114,6316343,Alta Floresta,BR,1592446165,0,70,-9.88,-56.09,70.520,1.51
133,133,4035715,Avarua,CK,1592446170,0,73,-21.21,-159.78,73.400,3.60


In [6]:
# Drop any null values

ideal_df.dropna()

ideal_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25 entries, 8 to 538
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  25 non-null     int64  
 1   City ID     25 non-null     int64  
 2   City        25 non-null     object 
 3   Country     23 non-null     object 
 4   Date        25 non-null     int64  
 5   Cloudiness  25 non-null     int64  
 6   Humidity    25 non-null     int64  
 7   Latitude    25 non-null     float64
 8   Longitude   25 non-null     float64
 9   Max Temp    25 non-null     float64
 10  Wind Speed  25 non-null     float64
dtypes: float64(4), int64(5), object(2)
memory usage: 2.3+ KB


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# store hotel_df

hotel_df = ideal_df

hotel_df["Hotel Name"] = ""


<ipython-input-7-54b155082ec2>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


In [8]:
# Set up google places API call

url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params ={
    "rankby": "distance",
    "type": "hotel",
    "key": g_key,
}
# Could not get it to work with "radius" = 5000 in my params for some reason - even upon manually entering these in a browser 
# upon doing more reading - I found that you cannot have location and radius

# Add Location column as a concatenation of lat,lng columns
location = hotel_df['Latitude'].astype(str) + "," + hotel_df['Longitude'].astype(str)
hotel_df['Location'] = location


# Set hotels as an empty list
hotels = []


# Use Location value to pull results from Places API for hotel nearest the given coordinates
for index,row in hotel_df.iterrows():
    # Get coordinates from df
    location = row['Location']
    # Add locations to params
    params['location'] = location
    response = requests.get(url, params=params).json()
    results = response['results'][0]['name']
    hotels.append(results)

# Add a new column to the hotel_df for the nearest hotel
hotel_df['Hotel Name'] = hotels

hotel_df.head()


    
    


<ipython-input-8-6e62ab509404>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Location'] = location
<ipython-input-8-6e62ab509404>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = hotels


,Unnamed: 0,City ID,City,Country,Date,Cloudiness,Humidity,Latitude,Longitude,Max Temp,Wind Speed,Hotel Name,Location
8,8,3356832,Henties Bay,NaN,1592446139,0,31,-22.12,14.28,73.994,0.89,LifeLink Emergency Rescue Services,"-22.12,14.28"
20,20,6620339,Karratha,AU,1592446142,0,35,-20.74,116.85,78.674,2.29,80 Mile Beach to Karratha Finish 2wd and 4wd,"-20.74,116.85"
51,51,2065594,Mount Isa,AU,1592446150,0,41,-20.73,139.50,78.800,6.20,Aboriginal Hostels Ltd.,"-20.73,139.5"
114,114,6316343,Alta Floresta,BR,1592446165,0,70,-9.88,-56.09,70.520,1.51,Oxalis Brindes,"-9.88,-56.09"
133,133,4035715,Avarua,CK,1592446170,0,73,-21.21,-159.78,73.400,3.60,Rarotonga Welding & Steel Construction,"-21.21,-159.78"


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]


In [10]:
# Add marker layer ontop of heat map
# Store humidity as weight
weight = hotel_df['Humidity']

# Store Heatmap in fig

fig = gmaps.figure()

# Create heat layer

heat_layer = gmaps.heatmap_layer(locations, weights=weight,dissipating = False,max_intensity = 10,point_radius=1)

fig.add_layer(heat_layer)
# Set marker location and info
markers = gmaps.marker_layer(locations,info_box_content = hotel_info)
# Add marker layer
fig.add_layer(markers)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))